<center> Итоговый проект обучения Skillfactory Data Science

# <center> Определение уязвимых групп населения

> ## Часть 1. Сбор и обработка данных

In [1]:
import pandas as pd
import numpy as np
import re
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Создадим словарь названий для унификации субъектов РФ
names = {'адыгея':'Республика Адыгея (Адыгея)','башкорт':'Республика Башкортостан',
'бурятия':'Республика Бурятия','дагестан':'Республика Дагестан','ингуш':'Республика Ингушетия',
'кабардино':'Кабардино-Балкарская Республика', 'карачаево':'Карачаево-Черкесская Республика',
'крым':'Республика Крым', 'республика саха':'Республика Саха (Якутия)', 'северная':'Республика Северная Осетия — Алания',
'татарстан':'Республика Татарстан (Татарстан)', 'тыва':'Республика Тыва (Тува)',
'чечня':'Чеченская Республика', 'чеченская':'Чеченская Республика',
'чуваш':'Чувашская Республика — Чувашия', 'кемеров':'Кемеровская область — Кузбасс',
'владимир':'Владимирская область', 'воронежская область':'Воронежская область',
'Bолгоградская область':'Волгоградская область',
'москва':'г. Москва', 'санкт':'г. Санкт-Петербург', 'севастополь':'г. Севастополь',
'еврейская':'Еврейская автономная область', 'ямало-ненецкий':'Ямало-Ненецкий автономный округ',
'ханты-мансийский':'Ханты-Мансийский автономный округ — Югра',
'ненецкий автономный округ (архангельская область)':'Ненецкий автономный округ',
'ненецкий авт. округ':'Ненецкий автономный округ', 'архангельская':'Архангельская область', 'чукотский авт. округ':'Чукотский автономный округ',
'тюменская':'Тюменская область', '\nненецкий':'Ненецкий автономный округ', 'ханты-мансийский ао':'Ханты-Мансийский автономный округ — Югра',
'ямало-ненецкий ао':'Ямало-Ненецкий автономный округ', 'тюменская область без автономного округа':'Тюменская область',
'ямало-ненецкий автономный округ':'Ямало-Ненецкий автономный округ'}


In [3]:
# Cоздадим функцию смены названия региона
def renaming(name):
    for n in names:
        if n in name.lower():
            return names[n]
    return name

>1.1 Среднедушевые денежные доходы (в месяц), руб

In [6]:
# Среднедушевые денежные доходы (в месяц), руб
cash_income_df = pd.read_excel('social_russia_data/cash_real_income_wages_2015_2020.xlsx')

In [7]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = cash_income_df[cash_income_df['region'].str.contains('федеральный округ|Федерация|район|числе')].index
cash_income_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
cash_income_df['region'] = cash_income_df['region'].apply(renaming)

# Назначим название региона индексом
cash_income_df.rename(columns={'region':"Регион"}, inplace=True)
cash_income_df["Регион"] = cash_income_df["Регион"].str.strip()
cash_income_df.set_index('Регион', inplace=True)

# Удалим дублирующиеся индексы 
dup_index = cash_income_df.index.duplicated()
cash_income_df = cash_income_df.loc[~dup_index]

print(cash_income_df.shape)
cash_income_df.head()

(85, 6)


,2015,2016,2017,2018,2019,2020
Регион,,,,,,
Белгородская область,28043.0,29799.0,30342.0,30778.0,32352.0,32841.0
Брянская область,23428.0,24006.0,25107.0,26585.0,28371.0,28596.0
Владимирская область,22712.0,22365.0,23554.0,23539.0,25358.0,25922.0
Воронежская область,29366.0,29284.0,29498.0,30289.0,32022.0,32078.0
Ивановская область,22297.0,23676.0,24860.0,24503.0,25794.0,26277.0


>1.2 Детская смертность

   *1.2.1 Детская смертность в сельской местности*

In [147]:
cm_rural_df = pd.read_excel('social_russia_data/child_mortality_rural_1990_2021.xls', skiprows=2)

In [148]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = cm_rural_df[cm_rural_df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район|числе')].index
cm_rural_df.drop(fo, axis = 0, inplace=True)

# Выявим строки содержащие в себе устаревшие субъекты и исключим их
cm_rural_df = cm_rural_df.dropna(axis=0)
# Унифицируем названия регионов
cm_rural_df['Unnamed: 0'] = cm_rural_df['Unnamed: 0'].apply(renaming)

# назначим название региона индексом
cm_rural_df.rename(columns={'Unnamed: 0':"Регион"}, inplace=True)
cm_rural_df["Регион"] = cm_rural_df["Регион"].str.strip()
cm_rural_df.set_index('Регион', inplace=True)

# Удалим дублирующиеся индексы 
dup_cmr_index = cm_rural_df.index.duplicated()
cm_rural_df = cm_rural_df.loc[~dup_cmr_index]

# Удалим лишние колонки Оба пола и Сельское население - везде
cm_rural_df.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)
# Удалим столбцы до 2016 года
old_d = list(map(lambda x: str(x),range(1990,2016)))
cm_rural_df.drop(old_d, axis = 1, inplace=True)


print(cm_rural_df.shape)
cm_rural_df.head()

(79, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,36.0,34.0,33.0,16.0,22.0,20.0
Брянская область,36.0,31.0,12.0,11.0,12.0,13.0
Владимирская область,28.0,17.0,22.0,15.0,10.0,14.0
Воронежская область,24.0,18.0,22.0,7.0,12.0,10.0
Ивановская область,15.0,4.0,7.0,9.0,3.0,4.0


*1.2.2 Детская смертность в городе*

In [149]:
cm_urban_df = pd.read_excel('social_russia_data/child_mortality_urban_1990_2021.xls', skiprows=2)

In [150]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = cm_urban_df[cm_urban_df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район|числе')].index
cm_urban_df.drop(fo, axis = 0, inplace=True)

# Выявим строки содержащие в себе устаревшие субъекты и исключим их
cm_urban_df = cm_urban_df.dropna(axis=0)
# Унифицируем названия регионов
cm_urban_df['Unnamed: 0'] = cm_urban_df['Unnamed: 0'].apply(renaming)

# назначим название региона индексом
cm_urban_df.rename(columns={'Unnamed: 0':"Регион"}, inplace=True)
cm_urban_df["Регион"] = cm_urban_df["Регион"].str.strip()
cm_urban_df.set_index('Регион', inplace=True)

# Удалим дублирующиеся индексы 
dup_cmu_index = cm_urban_df.index.duplicated()
cm_urban_df = cm_urban_df.loc[~dup_cmu_index]

# Удалим лишние колонки
cm_urban_df.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)
# Удалим столбцы до 2016 года
old_du = list(map(lambda x: str(x),range(1990,2016)))
cm_urban_df.drop(old_du, axis = 1, inplace=True)


print(cm_urban_df.shape)
cm_urban_df.head()


(81, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,72.0,43.0,40.0,23.0,25.0,34.0
Брянская область,67.0,65.0,33.0,26.0,29.0,14.0
Владимирская область,72.0,59.0,45.0,43.0,51.0,52.0
Воронежская область,96.0,90.0,77.0,74.0,58.0,73.0
Ивановская область,51.0,31.0,28.0,31.0,25.0,20.0


> 1.3 Число людей с инвалидностью по регионам, по месяцам, по возрастным группам.

In [34]:
disability_df = pd.read_csv('social_russia_data/disabled_total_by_age_2017_2022.csv')

In [35]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = disability_df[disability_df['region'].str.contains('федеральный округ|Федерация|район|Байконур')].index
disability_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
disability_df['region'] = disability_df['region'].apply(renaming)
disability_df.rename(columns={'region':"Регион"}, inplace=True)
disability_df["Регион"] = disability_df["Регион"].str.strip()

# Сгруппируем по дате и региону общее количество людей
disability_df['date'] = pd.to_datetime(disability_df['date']).dt.year
disability_df = disability_df.groupby(['Регион','date'])['total'].sum().unstack()
disability_df

date,2017,2018,2019,2020,2021,2022
Регион,,,,,,
Алтайский край,1987350.0,2120203.0,2100077.0,2060817.0,1986766.0,646296.0
Амурская область,816315.0,797050.0,785693.0,766768.0,733732.0,236444.0
Архангельская область,1056888.0,1038542.0,1031766.0,1016584.0,978993.0,316492.0
Астраханская область,533426.0,525601.0,523200.0,520687.0,511039.0,167664.0
Белгородская область,2625755.0,2522995.0,2446166.0,2359129.0,2252362.0,723760.0
...,...,...,...,...,...,...
Ямало-Ненецкий автономный округ,161326.0,161661.0,162704.0,165063.0,166606.0,56076.0
Ярославская область,1324880.0,1273883.0,1229274.0,1183293.0,1122990.0,358243.0
г. Москва,12682541.0,12486572.0,12263791.0,11900696.0,11409205.0,3689032.0


> 1.4 ВРП на душу населения, руб

In [40]:
grp_df = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', skiprows=2)

In [41]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = grp_df[grp_df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
grp_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
grp_df['Unnamed: 0'] = grp_df['Unnamed: 0'].apply(renaming)
# Назначим название области индексом
grp_df.rename(columns={'Unnamed: 0':"Регион"}, inplace=True)
grp_df["Регион"] = grp_df["Регион"].str.strip()
grp_df.set_index('Регион', inplace=True)
# Удалим дублирующиеся индексы 
dup_grp_index = grp_df.index.duplicated()
grp_df = grp_df.loc[~dup_grp_index]

# Удалим столбцы до 2016 года
old_grp = list(map(lambda x: str(x),range(1996,2016)))
grp_df.drop(old_grp, axis = 1, inplace=True)

print(grp_df.shape)
grp_df.head()

(85, 5)


,2016,2017,2018,2019,2020
Регион,,,,,
Белгородская область,501467.8,539720.5,588641.5,617426.5,646569.0
Брянская область,258752.7,280630.1,304547.1,332442.8,347204.5
Владимирская область,309713.7,325043.1,349856.4,394560.3,410443.6
Воронежская область,354657.9,374125.7,408140.7,431037.0,459629.5
Ивановская область,200504.4,208522.0,230325.5,249591.6,273821.5


> 1.5 Размер жилой площади в расчете на члена домохозяйства за 2020г , м2

In [42]:
housing_df = pd.read_excel('social_russia_data/housing_2020.xlsx')

In [43]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим их
fo = housing_df[housing_df['Регион'].str.contains('федеральный округ|Федерация|район')].index
housing_df.drop(fo, axis = 0, inplace=True)
# Унифицируем названия регионов
housing_df['Регион'] = housing_df['Регион'].apply(renaming)

# Назначим название области индексом
housing_df["Регион"] = housing_df["Регион"].str.strip()
housing_df.set_index('Регион', inplace=True)
# Оставим только интересующий признак
square_liv = housing_df[['Размер жилой площади в расчете на члена домохозяйства']]

diff = list(set(square_liv.index).difference(set(grp_df.index)))
square_liv.drop(diff, axis = 0, inplace=True)

print(housing_df.shape)
housing_df.head()

(85, 8)


,"Число домохозяйств, проживающих во всех типах жилых помещений","в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство
Регион,,,,,,,,
Белгородская область,100.0,86.3,11.7,2.0,0.0,25.1,17.8,2.6
Брянская область,100.0,94.7,4.6,0.6,0.0,30.5,24.1,3.2
Bладимирская область,100.0,81.8,15.8,2.5,0.0,24.2,16.1,2.2
Bоронежская область,100.0,87.0,11.4,1.6,0.1,29.0,19.9,2.6
Ивановская область,100.0,76.9,19.1,3.9,0.0,25.2,18.1,2.4


> 1.6 Заболеваемость

In [44]:
morbidity_df = pd.read_excel('social_russia_data/morbidity_2005_2020_age_disease.xls', skiprows=2)

In [45]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим 
fo = morbidity_df[morbidity_df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
morbidity_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
morbidity_df['Unnamed: 0'] = morbidity_df['Unnamed: 0'].apply(renaming)

# Назначим название области индексом
morbidity_df.rename(columns={'Unnamed: 0':"Регион",'Unnamed: 1':"Заболевания",'Unnamed: 2':"Группа"}, inplace=True)
morbidity_df["Регион"] = morbidity_df["Регион"].str.strip()
morbidity_df.set_index('Регион', inplace=True)
morbidity_df = morbidity_df[morbidity_df['Группа'] == 'Всего']

# Выделим датафрейм с общим показателем заболеваемости на 100тыс. человек населения за 2016г(только он попадает в выборку)
mor_all_df = morbidity_df[morbidity_df['Заболевания'].isin(['Все заболевания','Болезни системы кровообращения',\
    'Болезни органов дыхания','Болезни эндокринной системы, расстройства питания и нарушения обмена веществ'])].loc[:,['2016','Заболевания']]
mor_all_df = mor_all_df.dropna(axis=0)
mor_all_df = mor_all_df.groupby(['Регион','Заболевания'])['2016'].sum().unstack()
mor_all_df.rename(columns={'Все заболевания':'Заболеваемость на 100 тыс человек населения, чел.'}, inplace=True)

print(mor_all_df.shape)
mor_all_df.head()

(85, 4)


Заболевания,Болезни органов дыхания,Болезни системы кровообращения,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Заболеваемость на 100 тыс человек населения, чел."
Регион,,,,
Алтайский край,40416.4,5383.2,3885.0,112764.7
Амурская область,35518.0,2132.2,1147.3,83412.0
Архангельская область,44563.5,2452.0,1350.8,99842.1
Астраханская область,28101.5,2930.1,2029.4,64679.9
Белгородская область,27195.7,3995.8,813.6,74637.4


> 1.7 Процент людей, живущих за чертой бедности

In [46]:
poverty_percent_df = pd.read_csv('social_russia_data/poverty_percent_by_regions_1992_2020.csv')


In [47]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим 
fo = poverty_percent_df[poverty_percent_df['region'].str.contains('федеральный округ|Федерация|район')].index
poverty_percent_df.drop(fo, axis = 0, inplace=True)
# Унифицируем названия регионов
poverty_percent_df['region'] = poverty_percent_df['region'].apply(renaming)

# Назначим название области индексом
poverty_percent_df.rename(columns={'region':"Регион",'year':"Года"}, inplace=True)
poverty_percent_df["Регион"] = poverty_percent_df["Регион"].str.strip()
poverty_percent_df.set_index('Регион', inplace=True)

# Перегруппируем  в датафрейм и удалим лишние столбцы
poverty_percent_df.rename(columns={'year':"Года"}, inplace=True)
poverty_percent_df = poverty_percent_df.groupby(['Регион','Года'])['poverty_percent'].sum().unstack()
poverty_percent_df.drop(poverty_percent_df.columns[0:21],axis=1,inplace=True)

print(poverty_percent_df.shape)
poverty_percent_df.head()

(85, 5)


Года,2016,2017,2018,2019,2020
Регион,,,,,
Алтайский край,17.8,17.5,17.4,17.6,17.5
Амурская область,17.0,16.7,15.6,15.7,15.2
Архангельская область,30.1,28.2,26.0,26.3,25.1
Астраханская область,16.1,15.5,15.1,15.5,15.6
Белгородская область,8.1,7.8,7.5,7.8,7.2


> 1.8 Численность населения

In [55]:
population_df = pd.read_excel('social_russia_data\population.xlsx', header=1, usecols='A, C:Z')

In [58]:
population_df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population_df['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population_df.loc[i, 'region'] = population_df.loc[i-1, 'region']
population_df.dropna(subset=population_df.columns[1:], how='all', inplace=True)

# Выявим строки содержащие федеральный округ, федерация и район и исключим 
fo = population_df[population_df['region'].str.contains('федеральный округ|Федерация|район|сельсовет|все|поссовет|Раздел|пункты')].index
population_df.drop(fo, axis = 0, inplace=True)
# Унифицируем названия регионов
population_df['region'] = population_df['region'].apply(renaming)

# Назначим название области индексом
population_df.rename(columns={'region':"Регион"}, inplace=True)
population_df["Регион"] = population_df["Регион"].str.strip()
population_df.set_index('Регион', inplace=True)
# Удалим дублирующиеся индексы 
dup_pop_index = population_df.index.duplicated()
population_df = population_df.loc[~dup_pop_index]
# Выявим строки содержащие в себе лишние регионы и исключим их
old_d = list(set(population_df.index).difference(set(poverty_percent_df.index)))
population_df.drop(old_d, axis = 0, inplace=True)

# Удалим строки в названии столбцов
newcols = []
for col in population_df.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population_df.columns = pd.to_datetime(newcols).year

# Изменим тип столбцов с int32 на object
old_column_name = [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
new_column_name = ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

for i in range(len(old_column_name)):
    population_df = population_df.rename(
    columns={old_column_name[i]: new_column_name[i]})

print(population_df.shape)
population_df.head()

(85, 24)


,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Регион,,,,,,,,,,,,,,,,,,,,,
Белгородская область,1494868.0,1501699.0,1506976.0,1508137.0,1511899.0,1513860.0,1511662.0,1511715.0,1514153.0,1520102.0,...,1540985.0,1544108.0,1547936.0,1550137.0,1552865.0,1549876.0,1547418.0,1549151.0,1541259.0,1531917.0
Брянская область,1437471.0,1423178.0,1407965.0,1391430.0,1375004.0,1360249.0,1344132.0,1327652.0,1312748.0,1303343.0,...,1253666.0,1242599.0,1232940.0,1225741.0,1220530.0,1210982.0,1200187.0,1192491.0,1182682.0,1168771.0
Владимирская область,1592184.0,1575507.0,1558052.0,1539179.0,1520057.0,1509571.0,1497598.0,1486453.0,1475861.0,1466743.0,...,1421742.0,1413321.0,1405613.0,1397168.0,1389599.0,1378337.0,1365805.0,1358416.0,1342099.0,1323659.0
Воронежская область,2458558.0,2441337.0,2422371.0,2397111.0,2374461.0,2367457.0,2364932.0,2360912.0,2353805.0,2344413.0,...,2330377.0,2328959.0,2331147.0,2333477.0,2335408.0,2333768.0,2327821.0,2324205.0,2305608.0,2287678.0
Ивановская область,1210603.0,1194595.0,1178969.0,1161861.0,1144540.0,1131027.0,1116739.0,1101862.0,1089837.0,1081112.0,...,1048961.0,1043130.0,1036909.0,1029838.0,1023170.0,1014646.0,1004180.0,997135.0,987032.0,976918.0



> 1.9 Расходы на социальную политику, %

In [59]:
expense_soc_df = pd.read_excel('social_russia_data/welfare_expense_share_2015_2020.xlsx')

In [60]:
# выявим строки содержащие федеральный округ, федерация и район
fo = expense_soc_df[expense_soc_df['region'].str.contains('федеральный округ|Федерация|район')].index
expense_soc_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
expense_soc_df['region'] = expense_soc_df['region'].apply(renaming)

# назначим название области индексом
expense_soc_df.rename(columns={'region':"Регион"}, inplace=True)
expense_soc_df["Регион"] = expense_soc_df["Регион"].str.strip()
expense_soc_df.set_index('Регион', inplace=True)

print(expense_soc_df.shape)
expense_soc_df.head()

(85, 6)


,2015,2016,2017,2018,2019,2020
Регион,,,,,,
Белгородская область,11.3,11.8,16.4,15.6,13.8,16.1
Брянская область,22.0,26.6,24.7,23.7,20.7,23.1
Владимирская область,18.1,18.0,24.0,23.4,23.4,26.0
Воронежская область,15.2,15.4,22.8,20.9,19.0,21.6
Ивановская область,18.1,28.9,28.7,27.0,25.1,25.1


> 1.10 Оборот розничной торговли на душу населения, руб

In [65]:
turnover_retail_df = pd.read_excel('social_russia_data/retail_turnover_per_capita_2000_2021.xls', skiprows=2)

In [66]:
# Удалим ненужные столбцы и строки
turnover_retail_df.drop([0], axis=0, inplace=True)
turnover_retail_df.drop(turnover_retail_df.columns[1:18], axis=1,inplace=True)

# выявим строки содержащие федеральный округ, федерация и район
fo = turnover_retail_df[turnover_retail_df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
turnover_retail_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
turnover_retail_df['Unnamed: 0'] = turnover_retail_df['Unnamed: 0'].apply(renaming)

# назначим название области индексом
turnover_retail_df.rename(columns={'Unnamed: 0':"Регион"}, inplace=True)
turnover_retail_df["Регион"] = turnover_retail_df["Регион"].str.strip()
turnover_retail_df.set_index('Регион', inplace=True)

# Удалим дублирующиеся индексы 
dup_tr_index = turnover_retail_df.index.duplicated()
turnover_retail_df = turnover_retail_df.loc[~dup_tr_index]
# Выявим строки содержащие в себе лишние регионы и исключим их
old_d = list(set(turnover_retail_df.index).difference(set(poverty_percent_df.index)))
turnover_retail_df.drop(old_d, axis = 0, inplace=True)

print(turnover_retail_df.shape)
turnover_retail_df.head()

(85, 6)


,2016,2017,2018,2019,2020,2021
Регион,,,,,,
Белгородская область,192504,203298,217059,230704,237076,259151
Брянская область,179767,192775,209987,225818,225643,257679
Владимирская область,143643,153088,164519,175251,177680,207670
Воронежская область,208638,221302,236953,251882,252416,296665
Ивановская область,143568,153993,163290,178517,180314,211546


> 1.11 Распределение малоимущего населения по социально-демографическим группам

In [76]:
poverty_socdem_17_df = pd.read_excel('social_russia_data/poverty_socdem_2017.xls',skiprows=2)
poverty_socdem_18_df = pd.read_excel('social_russia_data/poverty_socdem_2018.xls',skiprows=2)
poverty_socdem_19_df = pd.read_excel('social_russia_data/poverty_socdem_2019.xls',skiprows=2)
poverty_socdem_20_df = pd.read_excel('social_russia_data/poverty_socdem_2020.xls',skiprows=2)


In [77]:
# функция обработки таблиц
# выявим строки содержащие федеральный округ, федерация и район, назначим название области индексом
def naming_df(df):
    fo = df[df['Unnamed: 0'].str.contains('федеральный округ|Федерация|район')].index
    df.drop(fo, axis = 0, inplace=True)
    df['Unnamed: 0'] = df['Unnamed: 0'].apply(renaming)
    df.rename(columns={'Unnamed: 0':"Регион"}, inplace=True)
    df["Регион"] = df["Регион"].str.strip()
    df.set_index('Регион', inplace=True)
    return df

# Удалим дублирующиеся индексы 
def duplicated_index(df):  
    dup_index = df.index.duplicated()
    df = df.loc[~dup_index]
    return df
# применим функцию к каждому датафрейму и заполним пустые значения
df_list = [poverty_socdem_17_df, poverty_socdem_18_df, poverty_socdem_19_df, poverty_socdem_20_df]

for i in df_list:
    naming_df(i)
    duplicated_index(i)
    i = i.fillna(0)
    
print(poverty_socdem_17_df.shape)
poverty_socdem_17_df.head()

(87, 4)


,Все население,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста
Регион,,,,
Белгородская область,100,43.4,11.8,44.8
Брянская область,100,42.9,4.9,52.2
Владимирская область,100,34.8,8.6,56.6
Воронежская область,100,38.6,5.9,55.6
Ивановская область,100,33.2,10.9,55.9


In [78]:
# Создадим признак % малоимущих детей
ps_17_deti = poverty_socdem_17_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_17_deti.rename(columns={'Дети в возрасте до 16 лет':'2017'},inplace=True)

ps_18_deti = poverty_socdem_18_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_18_deti.rename(columns={'Дети в возрасте до 16 лет':'2018'},inplace=True)

ps_19_deti = poverty_socdem_19_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_19_deti.rename(columns={'Дети в возрасте до 16 лет':'2019'},inplace=True)

ps_20_deti = poverty_socdem_20_df.loc[:, ['Дети в возрасте до 16 лет']]
ps_20_deti.rename(columns={'Дети в возрасте до 16 лет':'2020'},inplace=True)

poverty_socdem_deti = ps_17_deti.merge(ps_18_deti,on=['Регион'],how='left')
poverty_socdem_deti = poverty_socdem_deti.merge(ps_19_deti,on=['Регион'],how='left')
poverty_socdem_deti = poverty_socdem_deti.merge(ps_20_deti,on=['Регион'],how='left')
poverty_socdem_deti = duplicated_index(poverty_socdem_deti)
poverty_socdem_deti

,2017,2018,2019,2020
Регион,,,,
Белгородская область,43.4,44.8,0.0,NaN
Брянская область,42.9,40.3,37.5,44.8
Владимирская область,34.8,48.9,34.8,NaN
Воронежская область,38.6,32.6,45.2,28.5
Ивановская область,33.2,31.2,47.5,NaN
...,...,...,...,...
Амурская область,39.9,40.6,33.9,38.4
Магаданская область,32.2,30.7,45.8,NaN
Сахалинская область,36.4,32.9,44.9,NaN


In [79]:
# Создадим признак % малоимущих пенсионеров
ps_17_pens = poverty_socdem_17_df.loc[:, ['Население старше трудоспособного возраста']]
ps_17_pens.rename(columns={'Население старше трудоспособного возраста':'2017'},inplace=True)

ps_18_pens = poverty_socdem_18_df.loc[:, ['Население старше трудоспособного возраста']]
ps_18_pens.rename(columns={'Население старше трудоспособного возраста':'2018'},inplace=True)

ps_19_pens = poverty_socdem_19_df.loc[:, ['Население старше трудоспособного возраста']]
ps_19_pens.rename(columns={'Население старше трудоспособного возраста':'2019'},inplace=True)

ps_20_pens = poverty_socdem_20_df.loc[:, ['Население старше трудоспособного возраста']]
ps_20_pens.rename(columns={'Население старше трудоспособного возраста':'2020'},inplace=True)

poverty_socdem_pens = ps_17_pens.merge(ps_18_pens,on=['Регион'],how='left')
poverty_socdem_pens = poverty_socdem_pens.merge(ps_19_pens,on=['Регион'],how='left')
poverty_socdem_pens = poverty_socdem_pens.merge(ps_20_pens,on=['Регион'],how='left')
poverty_socdem_pens = duplicated_index(poverty_socdem_pens)
poverty_socdem_pens

,2017,2018,2019,2020
Регион,,,,
Белгородская область,11.8,12.9,0.0,NaN
Брянская область,4.9,1.8,5.1,3.7
Владимирская область,8.6,8.3,4.8,NaN
Воронежская область,5.9,8.0,5.4,2.3
Ивановская область,10.9,9.5,5.0,NaN
...,...,...,...,...
Амурская область,6.0,4.6,7.2,4.7
Магаданская область,14.8,11.5,0.0,NaN
Сахалинская область,6.4,4.3,12.1,NaN


In [80]:
# Создадим признак % малоимущих трудоспособных
ps_17_trud = poverty_socdem_17_df.loc[:, ['Население трудоспособного возраста']]
ps_17_trud.rename(columns={'Население трудоспособного возраста':'2017'},inplace=True)

ps_18_trud = poverty_socdem_18_df.loc[:, ['Население трудоспособного возраста']]
ps_18_trud.rename(columns={'Население трудоспособного возраста':'2018'},inplace=True)

ps_19_trud = poverty_socdem_19_df.loc[:, ['Население трудоспособного возраста']]
ps_19_trud.rename(columns={'Население трудоспособного возраста':'2019'},inplace=True)

ps_20_trud = poverty_socdem_20_df.loc[:, ['Население трудоспособного возраста']]
ps_20_trud.rename(columns={'Население трудоспособного возраста':'2020'},inplace=True)

poverty_socdem_trud = ps_17_trud.merge(ps_18_trud,on=['Регион'],how='left')
poverty_socdem_trud = poverty_socdem_trud.merge(ps_19_trud,on=['Регион'],how='left')
poverty_socdem_trud = poverty_socdem_trud.merge(ps_20_trud,on=['Регион'],how='left')
poverty_socdem_trud = duplicated_index(poverty_socdem_trud)
poverty_socdem_trud

,2017,2018,2019,2020
Регион,,,,
Белгородская область,44.8,42.3,0.0,NaN
Брянская область,52.2,57.9,57.4,51.6
Владимирская область,56.6,42.8,60.4,NaN
Воронежская область,55.6,59.4,49.4,69.2
Ивановская область,55.9,59.3,47.5,NaN
...,...,...,...,...
Амурская область,54.1,54.8,58.9,56.9
Магаданская область,53.0,57.8,54.2,NaN
Сахалинская область,57.1,62.8,43.0,NaN


> 1.12 Объем отгруженных товаров собственного производства, выполненных работ, услуг собственными силами, по видам деятельности (тыс. р. за год)

In [238]:
regional_prod_df = pd.read_csv('social_russia_data/regional_production_2017_2020.csv')

In [239]:
# Выявим строки содержащие федеральный округ, федерация и район и исключим 
fo = regional_prod_df[regional_prod_df['region'].str.contains('федеральный округ|Федерация|район|числе')].index
regional_prod_df.drop(fo, axis = 0, inplace=True)
# Унифицируем названия регионов
regional_prod_df['region'] = regional_prod_df['region'].apply(renaming)
# назначим название области индексом
regional_prod_df.rename(columns={'region':"Регион"}, inplace=True)
regional_prod_df["Регион"] = regional_prod_df["Регион"].str.strip()
regional_prod_df.set_index('Регион', inplace=True)
# переведем значения в рубли
# Выделим датафрейм с показателем по добыче полезных ископаемых
mining_df = (regional_prod_df[regional_prod_df['production_field'] == 'ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ'].loc[:,'2017':'2020'])*1000
# Выделим датафрейм с показателем по обрабатывающим производствам
manufacturing_df = (regional_prod_df[regional_prod_df['production_field'] == 'ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА'].loc[:,'2017':'2020'])*1000
# Выделим датафрейм с показателем по промышленности
industry_df = (regional_prod_df[regional_prod_df['production_field'] == 'Промышленное производство (промышленность)'].loc[:,'2017':'2020'])*1000
# Выделим датафрейм с показателем по обеспечения электроэнергией, газом и паром
supply_df = (regional_prod_df[regional_prod_df['production_field'] == 'ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА'].loc[:,'2017':'2020'])*1000

print(mining_df.shape)
display('Добыча полезных ископаемых', mining_df.head())
print(manufacturing_df.shape)
display('Обрабатывающие производства', manufacturing_df.head())
print(industry_df.shape)
display('Промышленность', industry_df.head())
print(supply_df.shape)
display('Обеспечение электроэнергией, газом и паром; кондиционирование воздухом', supply_df.head())


(85, 4)


'Добыча полезных ископаемых'

,2017,2018,2019,2020
Регион,,,,
Белгородская область,1.165883e+11,1.488634e+11,1.783899e+11,1.898038e+11
Брянская область,2.774277e+08,2.615495e+08,3.380273e+08,3.106659e+08
Владимирская область,4.003911e+09,5.005219e+09,5.425672e+09,4.965110e+09
Воронежская область,6.282742e+09,7.726219e+09,9.026583e+09,9.230600e+09
Ивановская область,7.546802e+08,9.718383e+08,1.237760e+09,1.170505e+09


(87, 4)


'Обрабатывающие производства'

...

'Промышленность'

,2017,2018,2019,2020
Регион,,,,
Белгородская область,7.833257e+11,8.954341e+11,9.390364e+11,9.741469e+11
Брянская область,2.147076e+11,2.454370e+11,2.809671e+11,2.834622e+11
Владимирская область,4.537824e+11,5.009853e+11,5.466535e+11,5.813260e+11
Воронежская область,5.295498e+11,5.690802e+11,6.409811e+11,7.182667e+11
Ивановская область,1.703785e+11,1.895272e+11,1.945330e+11,2.197861e+11


(87, 4)


'Обеспечение электроэнергией, газом и паром; кондиционирование воздухом'

,2017,2018,2019,2020
Регион,,,,
Белгородская область,2.703072e+10,2.680549e+10,2.654258e+10,2.678738e+10
Брянская область,1.606411e+10,1.773026e+10,1.770875e+10,1.787640e+10
Владимирская область,3.330818e+10,3.629512e+10,3.781567e+10,3.726466e+10
Воронежская область,8.820727e+10,9.961811e+10,1.103848e+11,1.377990e+11
Ивановская область,3.200315e+10,3.107706e+10,2.942105e+10,2.817499e+10


> 1.13 Число безработных, чел.

In [128]:
unemployed_df = pd.read_csv('social_russia_data/unemployed.csv')

In [129]:
# выявим строки содержащие федеральный округ, федерация и район
fo = unemployed_df[unemployed_df['Регион'].str.contains('федеральный округ|Федерация|район|Россия')].index
unemployed_df.drop(fo, axis = 0, inplace=True)
# Унифицируем названия регионов
unemployed_df['Регион'] = unemployed_df['Регион'].apply(renaming)
# назначим название области индексом
unemployed_df.rename(columns={'Регион':"Регион"}, inplace=True)
unemployed_df["Регион"] = unemployed_df["Регион"].str.strip()
unemployed_df.set_index('Регион', inplace=True)

print(unemployed_df.shape)
unemployed_df.tail()

(85, 22)


,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
Регион,,,,,,,,,,,,,,,,,,,,,
Чеченская Республика,94900.0,120800.0,85900.0,86300.0,87100.0,98100.0,105800.0,135900.0,174700.0,177900.0,...,194400.0,172100.0,254800.0,311200.0,95100.0,89500.0,106500.0,110100.0,149900.0,155600.0
Чувашская Республика — Чувашия,29000.0,36600.0,28300.0,31200.0,31900.0,33900.0,33200.0,34000.0,38300.0,38400.0,...,78000.0,53900.0,60500.0,58100.0,77400.0,64000.0,54800.0,63100.0,62500.0,61300.0
Чукотский АО,800.0,1400.0,1200.0,900.0,900.0,1100.0,1300.0,1100.0,1100.0,1400.0,...,1600.0,1700.0,1200.0,1300.0,1400.0,1200.0,1600.0,1600.0,2400.0,3700.0
Ямало-Ненецкий автономный округ,6600.0,7300.0,6100.0,6600.0,10000.0,8400.0,11300.0,10300.0,10300.0,11400.0,...,14300.0,19400.0,8600.0,16900.0,22300.0,19400.0,16900.0,20800.0,21300.0,25800.0
Ярославская обл.,39000.0,47400.0,35000.0,36400.0,44100.0,45200.0,36800.0,26400.0,30200.0,22700.0,...,55400.0,41100.0,23600.0,20500.0,27200.0,32300.0,41000.0,26800.0,48600.0,55300.0


In [130]:
# отсортируем столбцы
unemployed_df = unemployed_df.reindex(sorted(unemployed_df.columns), axis=1)
# переставим столбец регион в начало
unemployed_df['2000'] = unemployed_df['2021']
unemployed_df.drop(['2000'], axis=1, inplace = True)
unemployed_df.tail()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Регион,,,,,,,,,,,,,,,,,,,,,
Чеченская Республика,149900.0,110100.0,106500.0,89500.0,95100.0,311200.0,254800.0,172100.0,194400.0,238300.0,...,177900.0,174700.0,135900.0,105800.0,98100.0,87100.0,86300.0,85900.0,120800.0,94900.0
Чувашская Республика — Чувашия,62500.0,63100.0,54800.0,64000.0,77400.0,58100.0,60500.0,53900.0,78000.0,63400.0,...,38400.0,38300.0,34000.0,33200.0,33900.0,31900.0,31200.0,28300.0,36600.0,29000.0
Чукотский АО,2400.0,1600.0,1600.0,1200.0,1400.0,1300.0,1200.0,1700.0,1600.0,1500.0,...,1400.0,1100.0,1100.0,1300.0,1100.0,900.0,900.0,1200.0,1400.0,800.0
Ямало-Ненецкий автономный округ,21300.0,20800.0,16900.0,19400.0,22300.0,16900.0,8600.0,19400.0,14300.0,13400.0,...,11400.0,10300.0,10300.0,11300.0,8400.0,10000.0,6600.0,6100.0,7300.0,6600.0
Ярославская обл.,48600.0,26800.0,41000.0,32300.0,27200.0,20500.0,23600.0,41100.0,55400.0,50900.0,...,22700.0,30200.0,26400.0,36800.0,45200.0,44100.0,36400.0,35000.0,47400.0,39000.0


> 1.14 Рождаемость

In [137]:
birthrate_df = pd.read_csv('social_russia_data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')

In [138]:
# выявим строки содержащие федеральный округ, федерация и район
fo = birthrate_df[birthrate_df['Region'].str.contains('федеральный округ|Федерация|район')].index
birthrate_df.drop(fo, axis = 0, inplace=True)

# Унифицируем названия регионов
birthrate_df['Region'] = birthrate_df['Region'].apply(renaming)

# назначим название области индексом
birthrate_df.rename(columns={'Region':"Регион"}, inplace=True)
birthrate_df["Регион"] = birthrate_df["Регион"].str.strip()
birthrate_df.set_index('Регион', inplace=True)

birthrate_df = duplicated_index(birthrate_df)

# Выявим строки содержащие в себе лишние регионы и исключим их
old_d = list(set(birthrate_df.index).difference(set(poverty_percent_df.index)))
birthrate_df.drop(old_d, axis = 0, inplace=True)
birthrate_df = birthrate_df.drop(columns=birthrate_df.iloc[:, -6:])

# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in birthrate_df.columns:
        if str(year) in str(col):
            cols.append(col)
    birthrate_df[year] = birthrate_df[cols].sum(axis=1)
    birthrate_df.drop(columns=cols, inplace=True)
birthrate_df.drop(columns=2022, inplace=True)  # Неполный год
birthrate_df = birthrate_df.round(0).astype(int)
birthrate_df.columns = pd.to_datetime(birthrate_df.columns, format='%Y').year

# Изменим тип столбцов с int32 на object
old_name = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021]
new_name = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
                   '2015', '2016', '2017', '2018', '2019', '2020', '2021']

for i in range(len(old_name)):
    birthrate_df = birthrate_df.rename(
    columns={old_name[i]: new_name[i]})

print(birthrate_df.shape)
birthrate_df.head()

(85, 16)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Регион,,,,,,,,,,,,,,,,
Белгородская область,14269,15666,16810,16911,16669,16894,17913,17883,17822,17864,17261,15113,14318,13198,12352,12223
Брянская область,12042,13283,14319,14406,13773,13806,14376,13855,13658,14067,13379,11553,11124,10028,9480,9047
Владимирская область,13915,14679,15699,15520,15540,15644,16445,15777,15817,16284,15662,13395,12658,11158,10120,9780
Воронежская область,19208,20569,22422,23561,23820,23827,25374,24981,25332,25924,24938,22423,21382,19643,19161,18593
Ивановская область,9818,10617,11138,11330,11131,11035,11581,11769,11695,11761,11184,9877,9134,7951,7563,7327


> 1.15 Алкоголизм и наркомания

Единица измерения - количество акогольных/наркотических психозов на 100 тыс. населения.

In [145]:
alco_df = pd.read_excel('social_russia_data/drug_alco.xlsx',sheet_name='alco')
alco_1718 = pd.read_excel('social_russia_data/drug_alco.xlsx',sheet_name='alco1718')
drug_df = pd.read_excel('social_russia_data/drug_alco.xlsx',sheet_name='drugs')
drug_1718 = pd.read_excel('social_russia_data/drug_alco.xlsx',sheet_name='drug1718')

In [146]:
# выявим строки содержащие федеральный округ, федерация и район, назначим название области индексом
def renaming_df(data):
    fo = data[data['region'].str.contains('федеральный округ|Федерация|район|числе')].index
    data.drop(fo, axis = 0, inplace=True)
    data['region'] = data['region'].apply(renaming)
    data.rename(columns={'region':"Регион"}, inplace=True)
    data["Регион"] = data["Регион"].str.strip()
    data.set_index('Регион', inplace=True)
    return data
# применим функцию к каждому датафрейму и заполним пустые значения
list_df = [alco_df, alco_1718, drug_df, drug_1718]
for i in list_df:
    renaming_df(i)

alco_df = pd.concat([alco_df, alco_1718], axis=1).sort_index()
alco_df = duplicated_index(alco_df)
drug_df = pd.concat([drug_df, drug_1718], axis=1).sort_index()
drug_df = duplicated_index(drug_df)

print(alco_df.shape)
display('Алкоголизм', alco_df.head())
print(drug_df.shape)
display('Наркомания', drug_df.head())

(87, 14)


'Алкоголизм'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Регион,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7
Астраханская область,137.1,166.0,130.3,116.1,134.5,113.4,91.7,84.7,61.8,NaN,NaN,NaN,26.2,22.2
Белгородская область,99.6,93.2,84.0,92.3,94.3,86.4,72.6,74.8,66.9,56.8,50.7,46.1,45.2,41.3


(87, 14)


'Наркомания'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Регион,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2
Астраханская область,10.7,12.9,12.9,6.8,8.1,9.8,7.5,5.1,2.3,NaN,NaN,NaN,0.7,0.2
Белгородская область,4.5,9.5,9.5,4.2,3.9,4.8,4.0,2.3,2.4,1.9,2.0,1.6,2.3,3.2


> ## <center> Часть 2. Преобразование показателей и объединение в единый датафрейм

#### <center>*Вычисление удельных показателей на душу населения, где это возможно, и приведем показатели к средним значениям за 5 лет.*

In [240]:
# Среднегодовая численность населения, чел
population_mean = population_df.mean(axis=1)
# Среднедушевые денежные доходы, руб
cash_income_mean = cash_income_df.loc[:,2016:2020].mean(axis=1)
# Среднегодовая детская смертность на душу населения, чел
child_mortality = cm_rural_df + cm_urban_df 
child_mortality = child_mortality / population_df
child_mortality_mean = child_mortality.loc[:,'2016':'2020'].mean(axis=1)
# Среднегодовое число людей инвалидов , чел
disability = disability_df.loc[:,2017:2020].mean(axis=1)
disability_mean = disability / population_mean
# Среднегодовой валовой региональный продукт на душу населения, руб
grp_mean = grp_df.mean(axis=1)
# Среднегодовой процент людей, живущих за чертой бедности
poverty_percent_mean = poverty_percent_df.mean(axis=1)
# Среднегодовой процент расходов на социальную политику
expense_soc_mean = expense_soc_df.loc[:,2016.0:2020.0].mean(axis=1)
# Средегодовой оборот розничной торговли на душу населения, руб
turnover_retail_mean = turnover_retail_df.loc[:,'2016':'2020'].mean(axis=1)
# Средегодовой % малоимущих детей, доля
ps_deti_mean = poverty_socdem_deti.mean(axis=1)
# Средегодовой % малоимущих пенсионеров, доля
ps_pens_mean = poverty_socdem_pens.mean(axis=1)
# Средегодовой % малоимущих трудоспособных людей, доля
ps_trud_mean = poverty_socdem_trud.mean(axis=1)
# Средегодовой объем производимой добычи полезных ископаемых, руб на душу населения
mining_per = mining_df / population_df.loc[:,'2017':'2020']
mining_mean = mining_per.mean(axis=1).fillna(0)
mining_mean = duplicated_index(mining_mean)
# Средегодовой объем производимого обрабатывающего производства, руб на душу населения
manufacturing_per = manufacturing_df / population_df.loc[:,'2017':'2020']
manufacturing_mean = manufacturing_per.mean(axis=1).fillna(0)
manufacturing_mean = duplicated_index(manufacturing_mean)
# Средегодовой объем производимой промышленности, руб на душу населения
industry_per = industry_df / population_df.loc[:,'2017':'2020']
industry_mean = industry_per.mean(axis=1).fillna(0)
industry_mean = duplicated_index(industry_mean)
# Средегодовой объем производимой электрической энергии, газом и паром, руб на душу населения
supply_per = supply_df / population_df.loc[:,'2017':'2020']
supply_mean = supply_per.mean(axis=1).fillna(0)
supply_mean = duplicated_index(supply_mean)
# Среднегодовая численность безработных, чел 
unemp = unemployed_df.loc[:,'2016':'2020'].mean(axis=1)
unemployed_mean = unemp / population_mean
unemployed_mean = duplicated_index(unemployed_mean) 
# Среднегодовая рождаемость, чел 
birthrate_per = birthrate_df / population_df
borned_mean = birthrate_per.mean(axis=1)
# Среднегодовая заболеваемость впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза на 100 тыс.  населения, чел 
alco_mean = alco_df.loc[:,2016.0:2018.0].mean(axis=1)
# Среднегодовая заболеваемость впервые в жизни установленным диагнозом наркомании на 100 тыс.  населения, чел 
drug_mean = drug_df.loc[:,2016.0:2018.0].mean(axis=1)


> Объединим все признаки в итоговом датасете

In [242]:
result_region = pd.concat([population_mean, cash_income_mean, child_mortality_mean, disability_mean, grp_mean, 
                           poverty_percent_mean, expense_soc_mean, turnover_retail_mean, ps_deti_mean, ps_pens_mean, 
                           ps_trud_mean, mining_mean, manufacturing_mean, industry_mean, supply_mean, unemployed_mean, borned_mean, drug_mean, alco_mean,
                           mor_all_df, square_liv],axis=1).fillna(0)
result_region.rename(columns={0:'population', 1:'cash_income', 2:'child_mortality', 3:'disabled', 4:'grp', 
                              5:'poverty_%', 6:'Social_expenses_%', 7:'retail', 8:'poverty_deti_%', 
                              9:'poverty_pens_%', 10:'poverty_trud_%', 11:'mining', 12:'manufacturing', 
                              13:'industry', 14:'supply', 15:'unemployed', 16:'borned', 17:'drug', 18:'alco',
                              'Болезни органов дыхания':'respiratory_d',
                              'Болезни системы кровообращения':'blood_d', 
                              'Болезни эндокринной системы, расстройства питания и нарушения обмена веществ':'metabolic_d',
                              'Заболеваемость на 100 тыс человек населения, чел.':'all_disease',
                              'Размер жилой площади в расчете на члена домохозяйства':'square_liv'})
result_region.columns=['population', 'cash_income', 'child_mort', 'disabled', 'grp', 'poverty_%', 'Social_expenses_%',
                       'retail', 'poverty_deti_%', 'poverty_pens_%', 'poverty_trud_%', 'mining', 'manufacturing',
                       'industry', 'supply', 'unemployed', 'borned', 'drug', 'alco', 'respiratory_d', 'blood_d', 
                       'metabolic_d', 'all_disease', 'square_liv']
result_region = result_region.iloc[0:85]

result_region.to_csv('social_russia_data/result_regions.csv')
print(result_region.shape)
result_region

(85, 24)


,population,cash_income,child_mort,disabled,grp,poverty_%,Social_expenses_%,retail,poverty_deti_%,poverty_pens_%,...,supply,unemployed,borned,drug,alco,respiratory_d,blood_d,metabolic_d,all_disease,square_liv
Регион,,,,,,,,,,,,,,,,,,,,,
Белгородская область,1.528216e+06,31222.4,0.000044,1.628376,578765.06,7.68,14.74,216128.2,29.400000,8.233333,...,17286.678929,0.000000,0.010295,1.6,46.1,27195.7,3995.8,813.6,74637.4,17.8
Брянская область,1.288930e+06,26533.0,0.000053,1.001405,304715.44,13.92,23.76,206798.0,41.375000,3.875000,...,14387.151274,0.000000,0.010074,13.2,107.9,33294.4,3201.2,1641.5,77439.0,24.1
Владимирская область,1.449906e+06,24147.6,0.000052,1.044558,357923.42,13.20,22.96,162836.2,39.500000,7.233333,...,26355.518325,0.024677,0.010033,10.7,75.7,46461.1,2715.0,1265.2,94163.6,0.0
Воронежская область,2.352667e+06,30634.2,0.000041,1.107079,405518.16,8.94,19.94,234238.2,36.225000,5.400000,...,46790.873657,0.000000,0.009670,20.1,97.1,23444.6,4876.8,1589.2,55009.9,0.0
Ивановская область,1.076221e+06,25022.0,0.000040,0.880764,232553.00,14.36,26.96,163936.4,37.300000,8.466667,...,29865.357966,0.000000,0.009846,3.8,88.6,47678.7,2858.0,1676.5,88473.5,18.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Амурская область,8.434916e+05,31385.8,0.000058,0.938310,451290.98,16.04,24.74,216919.0,38.200000,5.625000,...,50694.970865,0.000000,0.012472,18.9,92.3,35518.0,2132.2,1147.3,83412.0,17.3
Магаданская область,1.627438e+05,60442.2,0.000040,0.370022,1392228.04,10.32,12.04,225678.0,36.233333,8.766667,...,117042.841726,0.000000,0.010973,11.6,161.9,29275.3,1453.4,1146.6,67012.9,13.8
Сахалинская область,5.125763e+05,54707.6,0.000050,0.504018,2029395.38,8.70,14.26,307109.8,38.066667,7.600000,...,37494.058034,0.000000,0.012408,15.4,141.4,38148.9,2228.6,1302.9,83995.5,17.2


> Вывод

Проведено считывание экономических и демографических данных из предоставленных файлов различного формата и из дополнительных источников. Результаты сохраняются в датасеты со стандартизованными названиями федеральных субъектов РФ (85 субъектов). Из полученных датасетов сформирована финальная таблица, которая далее используется в проекте.